In [1]:
pip install tonic-validate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
import openai


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
import os
from langchain.document_loaders import DirectoryLoader
from dotenv import load_dotenv
from langchain.llms import OpenAI
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import pandas as pd

In [12]:
#text splitting
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(texts[1]) 

page_content='According to a study from the University of Cambridge, at least half of developers’ efforts are spent debugging and not actively programming, which costs the software industry an estimated $312 billion per year. But so far, only a handful of code-generating AI systems have been made freely available to the public — reflecting the commercial incentives of the organizations building them (see: Replit).\n\nStarCoder, which by contrast is licensed to allow for royalty-free use by anyone, including corporations, was trained on over 80 programming languages as well as text from GitHub repositories, including documentation and programming notebooks. StarCoder integrates with Microsoft’s Visual Studio Code code editor and, like OpenAI’s ChatGPT, can follow basic instructions (e.g., “create an app UI”) and answer questions about code.' metadata={'source': 'new_articles/05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt'}


In [13]:
#load env variables(keys}
load_dotenv()

True

In [14]:
from openai import OpenAI
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
# embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
embedding_function = OpenAIEmbeddings(
api_key=OPENAI_API_KEY)

In [15]:
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=OpenAIEmbeddings())

In [33]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=vectordb.as_retriever())
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

In [17]:
#Load 10 questions and answers about the Paul Graham essays as a benchmark for how the RAG system should answer questions.
with open("question_and_answer_list.json", "r") as f:
    question_and_answer_list =json.load(f)

In [18]:
ex_q_and_a = question_and_answer_list[1]
ex_q_and_a["question"]

'What is ChatGPT, and how has it been used in various applications?'

In [21]:
ex_q_and_a["answer"]

'ChatGPT is a text-generating AI chatbot developed by OpenAI. It has been widely used for writing essays, code, and more based on short text prompts, enhancing productivity. Major brands have experimented with it for generating ad and marketing copy. OpenAI continually invests in ChatGPT, upgrading it to GPT-4, a more advanced language-writing model. The chatbot has been integrated into various applications, including search engines, customer service, and even an iPhone customization app called SuperChat.'

In [22]:
from tonic_validate import ValidateApi, ValidateScorer, Benchmark, LLMResponse
from tonic_validate.metrics import AnswerConsistencyMetric, AugmentationAccuracyMetric

In [23]:
from tonic_validate.validate_scorer import ValidateScorer 
# metrics
from tonic_validate.metrics.answer_consistency_metric import AnswerConsistencyMetric
from tonic_validate.metrics.answer_similarity_metric import AnswerSimilarityMetric
from tonic_validate.metrics.augmentation_accuracy_metric import AugmentationAccuracyMetric
from tonic_validate.metrics.augmentation_precision_metric import AugmentationPrecisionMetric
from tonic_validate.metrics.retrieval_precision_metric import RetrievalPrecisionMetric
# llm utils
from tonic_validate.classes.llm_response import LLMResponse
from tonic_validate.classes.benchmark import BenchmarkItem

In [24]:
metrics = [
    AnswerSimilarityMetric(),
    RetrievalPrecisionMetric(),
    AugmentationAccuracyMetric(),
    AugmentationPrecisionMetric(),
    AnswerConsistencyMetric()
]
# can use an OpenAI chat completion model
# llm_evaluator = "gpt-3.5-turbo"
llm_evaluator = "gpt-4-1106-preview"
validate_scorer = ValidateScorer(
    metrics, llm_evaluator
)


For one benchMark

In [34]:
# example BenchmarkItem
question = ex_q_and_a["question"]
reference_answer = ex_q_and_a["answer"]
benchmark_item = BenchmarkItem(
    question=question,
    answer=reference_answer
)

# example LLMResponse
llm_answer = response
context_list = [document.page_content for document in matching_docs]
llm_response = LLMResponse(
    llm_answer=llm_answer,
    llm_context_list=context_list,
    benchmark_item=benchmark_item
)

responses = [llm_response]

response_scores = validate_scorer.score_run(responses)

In [35]:
response_scores.run_data[0].scores

{'answer_similarity': 4.0,
 'retrieval_precision': 1.0,
 'augmentation_accuracy': 0.5,
 'augmentation_precision': 0.5,
 'answer_consistency': 1.0}

In [30]:
# responses = []

# for q_and_a in question_and_answer_list:
#     query = ex_q_and_a["question"]
#     matching_docs =vectordb.similarity_search(query)
#     response=chain.run(input_documents=matching_docs, question=query)
    
#     benchmark_item = BenchmarkItem(
#         question=q_and_a["question"],
#         answer=q_and_a["answer"]
#     )

#     llm_response = LLMResponse(
#         llm_answer=response,
#         llm_context_list=[document.page_content for document in matching_docs],
#         benchmark_item=benchmark_item
#     )

#     responses.append(llm_response)

In [31]:
# response_scores = validate_scorer.score_run(responses)

In [36]:
validate_api = ValidateApi("lYNVCKKhUtssnDKSQGX48HkK4Mr_62dr9Hj7U0_K98c")
validate_api.upload_run("944f02fe-c106-45c0-af5d-74e4bd0518b7", response_scores)

In [17]:
# def make_scores_df(response_scores):
#     scores_df = {
#         "question": [],
#         "reference_answer": [],
#         "llm_answer": [],
#         "retrieved_context": []
#     }
#     for score_name in response_scores.overall_scores:
#         scores_df[score_name] = []
#     for data in response_scores.run_data:
#         scores_df["question"].append(data.reference_question)
#         scores_df["reference_answer"].append(data.reference_answer)
#         scores_df["llm_answer"].append(data.llm_answer)
#         scores_df["retrieved_context"].append(data.llm_context)
#         for score_name, score in data.scores.items():
#             scores_df[score_name].append(score)
#     return pd.DataFrame(scores_df)
            


In [18]:
# scores_df = make_scores_df(response_scores)

In [21]:
# scores_df.head()

question  \
0  What does Pando plan to use the $30 million ra...   

                                    reference_answer  \
0  Pando intends to use the funds for expanding i...   

                                          llm_answer  \
0  Pando plans to use the $30 million raised in i...   

                                   retrieved_context  answer_similarity  \
0  [Signaling that investments in the supply chai...                4.0   

   retrieval_precision  augmentation_accuracy  augmentation_precision  \
0                  0.5                   0.25                     0.5   

   answer_consistency  
0                 1.0